In [0]:
import numpy as np
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My\ Drive/465data/

/content/drive/My Drive/465data


In [0]:
ls

32ptx_data_n                         HAM10000_images_part_2labels_np.npy
HAM10000_images_part_2images_np.npy  images_np_32.npy


In [0]:
dataset = pd.read_csv("64ptx_data_n")


In [5]:
print("Shape of dataset", dataset.shape)
dataset.head()

Shape of dataset (9993, 12296)


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,12255,12256,12257,12258,12259,12260,12261,12262,12263,12264,12265,12266,12267,12268,12269,12270,12271,12272,12273,12274,12275,12276,12277,12278,12279,12280,12281,12282,12283,12284,12285,12286,12287,image_id,lesion_id,dx,dx_type,age,sex,localization
0,0,229,132,135,230,126,134,230,124,126,234,130,130,234,132,133,229,128,125,231,128,129,238,134,138,240,139,146,237,130,137,238,131,139,240,136,143,239,135,144,...,225,143,130,224,143,128,221,143,131,224,149,135,223,148,136,222,146,134,223,149,136,222,150,138,219,149,135,218,146,135,212,142,131,ISIC_0024306,HAM_0000550,nv,follow_up,45.0,male,trunk
1,1,216,120,133,218,124,139,218,124,134,220,123,135,217,120,135,218,120,135,221,120,134,220,123,134,218,124,134,220,122,128,219,121,133,218,121,132,221,126,138,...,218,141,145,214,140,146,221,146,151,224,147,152,218,144,150,214,142,146,211,140,143,211,138,141,214,139,145,204,133,136,203,132,134,ISIC_0024307,HAM_0003577,nv,follow_up,50.0,male,lower extremity
2,2,246,172,180,246,170,176,246,173,180,246,174,182,247,174,184,247,175,184,247,175,184,247,177,184,248,181,186,248,183,190,248,184,191,248,182,189,248,181,187,...,215,135,137,216,137,143,214,135,139,212,134,138,210,129,143,210,133,147,215,143,153,219,150,158,219,151,160,217,148,156,216,145,155,ISIC_0024308,HAM_0001477,nv,follow_up,55.0,female,trunk
3,3,207,118,134,211,120,134,207,116,131,208,117,134,212,123,138,211,123,139,211,127,140,213,125,141,214,127,142,213,126,141,213,124,142,213,123,139,212,123,139,...,201,132,129,198,128,122,195,124,117,196,127,120,194,121,119,193,121,119,193,123,118,190,120,119,189,118,121,189,120,124,189,124,127,ISIC_0024309,HAM_0000484,nv,follow_up,40.0,male,trunk
4,4,71,32,37,70,32,39,63,25,30,94,45,51,165,105,115,195,133,140,208,149,153,208,144,152,210,145,160,210,146,163,209,146,157,214,154,164,214,159,167,...,206,137,114,204,127,103,204,122,118,205,132,137,201,120,104,203,124,129,194,132,146,159,102,98,93,47,51,70,30,42,73,33,44,ISIC_0024310,HAM_0003350,mel,histo,60.0,male,chest


In [6]:
X = dataset.drop(columns = ["Unnamed: 0","dx","image_id","lesion_id","dx","dx_type","age","sex","localization"])
print("Shape of X", X.shape)
X.head()

Shape of X (9993, 12288)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,12248,12249,12250,12251,12252,12253,12254,12255,12256,12257,12258,12259,12260,12261,12262,12263,12264,12265,12266,12267,12268,12269,12270,12271,12272,12273,12274,12275,12276,12277,12278,12279,12280,12281,12282,12283,12284,12285,12286,12287
0,229,132,135,230,126,134,230,124,126,234,130,130,234,132,133,229,128,125,231,128,129,238,134,138,240,139,146,237,130,137,238,131,139,240,136,143,239,135,144,235,...,135,225,144,129,223,138,124,225,143,130,224,143,128,221,143,131,224,149,135,223,148,136,222,146,134,223,149,136,222,150,138,219,149,135,218,146,135,212,142,131
1,216,120,133,218,124,139,218,124,134,220,123,135,217,120,135,218,120,135,221,120,134,220,123,134,218,124,134,220,122,128,219,121,133,218,121,132,221,126,138,221,...,154,225,149,155,221,143,148,218,141,145,214,140,146,221,146,151,224,147,152,218,144,150,214,142,146,211,140,143,211,138,141,214,139,145,204,133,136,203,132,134
2,246,172,180,246,170,176,246,173,180,246,174,182,247,174,184,247,175,184,247,175,184,247,177,184,248,181,186,248,183,190,248,184,191,248,182,189,248,181,187,248,...,151,221,147,151,216,140,145,215,135,137,216,137,143,214,135,139,212,134,138,210,129,143,210,133,147,215,143,153,219,150,158,219,151,160,217,148,156,216,145,155
3,207,118,134,211,120,134,207,116,131,208,117,134,212,123,138,211,123,139,211,127,140,213,125,141,214,127,142,213,126,141,213,124,142,213,123,139,212,123,139,214,...,129,199,128,128,199,128,127,201,132,129,198,128,122,195,124,117,196,127,120,194,121,119,193,121,119,193,123,118,190,120,119,189,118,121,189,120,124,189,124,127
4,71,32,37,70,32,39,63,25,30,94,45,51,165,105,115,195,133,140,208,149,153,208,144,152,210,145,160,210,146,163,209,146,157,214,154,164,214,159,167,215,...,131,214,153,159,209,149,145,206,137,114,204,127,103,204,122,118,205,132,137,201,120,104,203,124,129,194,132,146,159,102,98,93,47,51,70,30,42,73,33,44


In [7]:
Y = dataset["dx"]
Y = pd.get_dummies(Y)
print("Shape of Y", Y.shape)
Y.head()

Shape of Y (9993, 7)


,akiec,bcc,bkl,df,mel,nv,vasc
0,0,0,0,0,0,1,0
1,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0


In [8]:
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15, random_state = 42)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train_flat shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test_flat shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 8494
number of test examples = 1499
X_train_flat shape: (8494, 12288)
Y_train shape: (8494, 7)
X_test_flat shape: (1499, 12288)
Y_test shape: (1499, 7)


In [0]:
X_train = X_train.values
Y_train = Y_train.values

In [0]:
def LRModel1(input_shape = (3072,), classes = 7):
    X_input = Input(input_shape)
    X = BatchNormalization()(X_input)
    X = Dense(input_shape, activation='sigmoid', name = "sigmoid_activation", kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dense(classes, input_shape=input_shape, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    model = Model(inputs = X_input, outputs = X, name='LogisticRegression')
    return model

def LRModel2(input_shape = (3072,), classes = 7):
    X_input = Input(input_shape)
    X = BatchNormalization()(X_input)
    X = Dense(classes, input_shape=input_shape, activation='sigmoid', name = "sigmoid_activation", kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation("softmax")(X)
    model = Model(inputs = X_input, outputs = X, name='LogisticRegression')
    return model

In [0]:
model = LRModel2(input_shape = (12288,), classes = 7)

In [0]:
from keras.optimizers import SGD, Adam
adam = Adam(lr= 0.0005)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, Y_train, epochs = 200, batch_size = 32)

Epoch 1/200
8494/8494 [==============================] - 4s 471us/step - loss: 1.6551 - accuracy: 0.5039
Epoch 2/200
8494/8494 [==============================] - 4s 435us/step - loss: 1.4628 - accuracy: 0.5637
Epoch 3/200
8494/8494 [==============================] - 4s 431us/step - loss: 1.4492 - accuracy: 0.5812
Epoch 4/200
8494/8494 [==============================] - 4s 439us/step - loss: 1.4425 - accuracy: 0.5963
Epoch 5/200
8494/8494 [==============================] - 4s 453us/step - loss: 1.4377 - accuracy: 0.5834
Epoch 6/200
8494/8494 [==============================] - 4s 448us/step - loss: 1.4317 - accuracy: 0.5934
Epoch 7/200
8494/8494 [==============================] - 4s 455us/step - loss: 1.4293 - accuracy: 0.6164
Epoch 8/200
8494/8494 [==============================] - 4s 442us/step - loss: 1.4295 - accuracy: 0.6317
Epoch 9/200
8494/8494 [==============================] - 4s 434us/step - loss: 1.4286 - accuracy: 0.6334
Epoch 10/200
8494/8494 [==============================]

In [0]:
X_test = X_test.values
Y_test = Y_test.values

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score, average_precision_score, f1_score
y_pred_test = model.predict(X_test)
y_pred_train = model.predict(X_train)
conf_m_test = confusion_matrix(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
y_test_precision = average_precision_score(Y_test, y_pred_test)
y_test_acc1 = accuracy_score(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
y_test_f1 = f1_score(Y_test.argmax(axis=1), y_pred_test.argmax(axis=1), average = 'weighted')
y_train_f1 = f1_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1), average = 'weighted')
print("Precision of the Logistic Regression model in test set: " + str(y_test_precision) + "\n Accuracy of the Logistic Regression Modelin test set: " + str(y_test_acc1))
print("f1 of the Logistic Regression model in test set: " + str(y_test_f1))

Precision of the Logistic Regression model in test set: 0.3113456721226445
 Accuracy of the Logistic Regression Modelin test set: 0.6797865243495663
f1 of the Logistic Regression model in test set: 0.6287490743685807


In [0]:
y_train_precision = average_precision_score(Y_train, y_pred_train)
y_train_acc1 = accuracy_score(Y_train.argmax(axis=1), y_pred_train.argmax(axis=1))
print("Precision of the Logistic Regression model in test set: " + str(y_test_precision) + "\n Accuracy of the Logistic Regression Modelin test set: " + str(y_test_acc1))

Precision of the Logistic Regression model in test set: 0.25005542865124253
 Accuracy of the Logistic Regression Modelin test set: 0.6490993995997332


In [23]:
from keras.models import Sequential
from keras.layers import Dropout
# Set CNN model
# Our system of layers => [[Conv2D -> relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (64, 64, 3)
num_classes = 7


model_cnn = Sequential()
model_cnn.add(Conv2D(10, kernel_size = (3, 3), activation = 'relu', padding = 'Same', input_shape = input_shape))
model_cnn.add(MaxPooling2D(pool_size = (2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Conv2D(20, kernel_size = (5, 5), activation = 'relu', padding = 'Same'))
model_cnn.add(MaxPooling2D(pool_size = (3, 3)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Conv2D(30, kernel_size = (7, 7), activation = 'relu', padding = 'Same'))
model_cnn.add(MaxPooling2D(pool_size = (4, 4)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten())
model_cnn.add(Dense(120, activation='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(64, activation ='relu'))
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(num_classes, activation='softmax'))
model_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 10)        280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 10)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 10)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 20)        5020      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 30)       

In [0]:
model_cnn.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
X_train3D = np.reshape(X_train, (X_train.shape[0], 64, 64, 3))
X_test3D = np.reshape(X_test, (X_test.shape[0], 64, 64, 3))

In [0]:
model_cnn.fit(X_train3D, Y_train, batch_size = 32, epochs = 200)

Epoch 1/200
8494/8494 [==============================] - 41s 5ms/step - loss: 2.3068 - accuracy: 0.6563
Epoch 2/200
8494/8494 [==============================] - 41s 5ms/step - loss: 1.0070 - accuracy: 0.6665
Epoch 3/200
8494/8494 [==============================] - 41s 5ms/step - loss: 0.9763 - accuracy: 0.6664
Epoch 4/200
8494/8494 [==============================] - 40s 5ms/step - loss: 0.9571 - accuracy: 0.6706
Epoch 5/200
8494/8494 [==============================] - 40s 5ms/step - loss: 0.9391 - accuracy: 0.6668
Epoch 6/200
8494/8494 [==============================] - 41s 5ms/step - loss: 0.9374 - accuracy: 0.6700
Epoch 7/200
8494/8494 [==============================] - 40s 5ms/step - loss: 0.9165 - accuracy: 0.6701
Epoch 8/200
8494/8494 [==============================] - 40s 5ms/step - loss: 0.9190 - accuracy: 0.6704
Epoch 9/200
8494/8494 [==============================] - 40s 5ms/step - loss: 0.8981 - accuracy: 0.6728
Epoch 10/200
8494/8494 [==============================] - 40s 5m